### Logistic Regression model with only loudness

#### Modules

In [88]:
import pandas as pd
import numpy as np

from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix

#### Prepare the Data 

In [89]:
# read data
data = pd.read_pickle('merged_data.pkl') 

# identfiy numerical volumns
columns = ['danceability','energy','key','mode','loudness',
 'speechiness','acousticness','instrumentalness',
 'liveness','valence','tempo','time_signature','disc_number','duration_ms','is_hit']

df = data[columns] # get numerical columns
df.head(1)

,danceability,energy,key,mode,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,disc_number,duration_ms,is_hit
0,0.67,0.428,3,0,-10.161,0.0369,0.537,0.000003,0.118,0.326,100.033,4,1,289596,0


In [90]:
X = df.drop('is_hit', axis=1) # features for model
y = df['is_hit'] # target variable

In [91]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)

/Users/rohanmalhotra/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int32, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/rohanmalhotra/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DataConversionWarning: Data with input dtype int32, float64 were all converted to float64 by StandardScaler.
  after removing the cwd from sys.path.


In [92]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 0)

#### Picking the best model

In [93]:
from sklearn.feature_selection import RFECV

In [94]:
model = RFECV(LogisticRegression(solver = 'lbfgs',max_iter = 300), scoring = "accuracy", cv = 5)



In [95]:
model.fit(X,y)
print(model.n_features_) # How many features in the model
print(model.ranking_) # Best features of the model to use 

1
[ 7  2 13 11  1 14 10  3  5  4  8 12  6  9]


#### Best Model would be with one feature being loudness

In [99]:
model.score(X,y)

0.8873247945867568

In [97]:
# cross-validation
rfc_cv_score = cross_val_score(model, X, y, cv=10, scoring="roc_auc")

# predictions
y_predict = model.predict(x_test)

In [98]:
print("=== Confusion Matrix ===")
# print(confusion_matrix(y_test, y_predict))
x = pd.crosstab(y_test, y_predict, rownames=['Actual'], colnames=['Predicted'])
print(x)
print('\n')
print("=== Classification Report ===")
print(classification_report(y_test, y_predict))
print("=== All AUC Scores ===")
print(rfc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Logistic Regression ", rfc_cv_score.mean())

=== Confusion Matrix ===
Predicted      0
Actual          
0          13230
1           1667


=== Classification Report ===
              precision    recall  f1-score   support

           0       0.89      1.00      0.94     13230
           1       0.00      0.00      0.00      1667

   micro avg       0.89      0.89      0.89     14897
   macro avg       0.44      0.50      0.47     14897
weighted avg       0.79      0.89      0.84     14897

=== All AUC Scores ===
[0.49758035 0.44490447 0.54691107 0.55186211 0.53312204 0.48308028
 0.51814407 0.51402503 0.58169235 0.51724596]


=== Mean AUC Score ===
Mean AUC Score - Logistic Regression  0.5188567726604852


/Users/rohanmalhotra/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
